In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import Lasso

In [2]:
# Define parameters for LASSO
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10]
#max_iter = [500, 1000, 10000, 100000]
selection = ['cyclic', 'random']
fit_intercept = [True, False]
tol = [0.0001, 0.001, 0.01, 0.1, 1, 10]
warm_start = [True, False]
#positive = [True, False]
#random_state = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
copy_X = [True, False]

# Create the random grid
random_grid = {'alpha': alpha,
               #'max_iter': max_iter,
               'selection': selection,
               'fit_intercept': fit_intercept,
               'tol': tol,
               'warm_start': warm_start,
               #'positive': positive,
               #'random_state': random_state,
               'copy_X': copy_X}

In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

scorer_mase= make_scorer(mean_absolute_scaled_error, greater_is_better=False)


In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_30min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date']) #,format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
#Feature engineering
# Create a new column for the time
data['time'] = [x for x in range(0, len(data))]

data['hour_of_day'] = data['date'].dt.hour
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_month'] = data['date'].dt.day
data['month_of_year'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# make a weekend column
data['weekend'] = 0
data.loc[data['day_of_week'] == 5, 'weekend'] = 1
data.loc[data['day_of_week'] == 6, 'weekend'] = 1

#make a monday column
data['monday'] = 0
data.loc[data['day_of_week'] == 0, 'monday'] = 1

#make a tuesday column
data['tuesday'] = 0
data.loc[data['day_of_week'] == 1, 'tuesday'] = 1

#make a wednesday column
data['wednesday'] = 0
data.loc[data['day_of_week'] == 2, 'wednesday'] = 1

#make a thursday column
data['thursday'] = 0
data.loc[data['day_of_week'] == 3, 'thursday'] = 1

#make a friday column
data['friday'] = 0
data.loc[data['day_of_week'] == 4, 'friday'] = 1

#make a saturday column
data['saturday'] = 0
data.loc[data['day_of_week'] == 5, 'saturday'] = 1

#make a sunday column
data['sunday'] = 0
data.loc[data['day_of_week'] == 6, 'sunday'] = 1


# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# Define the feature columns and the target column
feature_cols = ['time', 'hour_of_day', 'day_of_week', 'day_of_month', 'month_of_year', 'year', 'weekend', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
target_col = 'y'

# Drop nan values
data = data.dropna()

# Rename column count to y
data = data.rename(columns={'count': 'y'})

In [6]:
# Set y to the last column
cols = list(data.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('y')) #Remove y from list
data = data[cols+['y']] #Create new dataframe with columns in the order you want

# drop the date column
train_data = np.delete(data, 0, 1) 

# Split the data into X and y
X_train, y_train = train_data[:, :-1], train_data[:, -1]


In [7]:
#vgl: https://lightrun.com/answers/scikit-learn-scikit-learn-grid_search-feeding-parameters-to-scorer-functions

# X and y to pandas dataframe
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)


In [8]:
# Set Cross Validation to 5 iterations
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Instantiate model
model = Lasso()

search = RandomizedSearchCV(estimator = model, 
                               param_distributions = random_grid, 
                               n_iter = 25, 
                               cv = tscv,
                               refit=True, 
                               verbose=3, 
                               random_state=42, 
                               n_jobs = -1, 
                               scoring=scorer_mase, #make_scorer(scorer_mase, greater_is_better=True), #'neg_root_mean_squared_error', #
                               error_score=np.nan)


In [10]:
# Fit the random search model
search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.197 total time=   0.0s
[CV 4/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-1.153 total time=   0.0s
[CV 2/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-1.318 total time=   0.0s
[CV 3/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-1.197 total time=   0.0s
[CV 5/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-1.068 total time=   0.0s
[CV 1/5] END alpha=0.1, copy_X=False, fit_intercept=True, selection=cyclic, tol=10, warm_start=True;, score=-1.224 total time=   0.0s
[CV 2/5] END alpha=0.1, copy_X=False, fit_intercept=True, selection=cyclic, tol=10, warm_start=True;, score=-1.324 total time=  

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.864e+04, tolerance: 4.004e+03
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.343e+05, tolerance: 8.221e+03
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

[CV 4/5] END alpha=0.1, copy_X=False, fit_intercept=False, selection=random, tol=1, warm_start=True;, score=-1.285 total time=   0.0s
[CV 1/5] END alpha=1, copy_X=False, fit_intercept=False, selection=random, tol=0.001, warm_start=True;, score=-1.223 total time=   0.0s
[CV 5/5] END alpha=0.1, copy_X=False, fit_intercept=False, selection=random, tol=1, warm_start=True;, score=-1.249 total time=   0.0s
[CV 2/5] END alpha=1, copy_X=False, fit_intercept=False, selection=random, tol=0.001, warm_start=True;, score=-1.335 total time=   0.0s
[CV 3/5] END alpha=1, copy_X=False, fit_intercept=False, selection=random, tol=0.001, warm_start=True;, score=-1.208 total time=   0.0s
[CV 3/5] END alpha=0.0001, copy_X=True, fit_intercept=False, selection=cyclic, tol=0.01, warm_start=False;, score=-1.195 total time=   0.2s
[CV 2/5] END alpha=1, copy_X=False, fit_intercept=True, selection=random, tol=0.001, warm_start=False;, score=-1.335 total time=   0.0s
[CV 5/5] END alpha=1, copy_X=False, fit_intercep

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.618e+05, tolerance: 1.605e+04
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.309e+05, tolerance: 2.054e+04
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

[CV 3/5] END alpha=0.01, copy_X=False, fit_intercept=False, selection=random, tol=0.01, warm_start=False;, score=-1.194 total time=   0.1s
[CV 1/5] END alpha=0.01, copy_X=False, fit_intercept=False, selection=cyclic, tol=0.0001, warm_start=False;, score=-1.197 total time=   0.0s
[CV 5/5] END alpha=0.01, copy_X=False, fit_intercept=False, selection=random, tol=0.01, warm_start=False;, score=-1.066 total time=   0.1s
[CV 1/5] END alpha=0.1, copy_X=False, fit_intercept=True, selection=random, tol=1, warm_start=True;, score=-1.202 total time=   0.0s
[CV 3/5] END alpha=10, copy_X=True, fit_intercept=False, selection=random, tol=1, warm_start=True;, score=-1.798 total time=   0.0s
[CV 2/5] END alpha=0.001, copy_X=False, fit_intercept=True, selection=cyclic, tol=1, warm_start=False;, score=-1.318 total time=   0.0s
[CV 5/5] END alpha=0.0001, copy_X=True, fit_intercept=False, selection=cyclic, tol=0.01, warm_start=False;, score=-1.066 total time=   0.4s
[CV 2/5] END alpha=0.01, copy_X=False, f

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.865e+04, tolerance: 4.004e+04
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.848e+04, tolerance: 4.004e+01
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

[CV 1/5] END alpha=0.0001, copy_X=False, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.196 total time=   0.1s
[CV 1/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.0001, warm_start=True;, score=-1.197 total time=   0.1s
[CV 2/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.0001, warm_start=True;, score=-1.318 total time=   0.1s
[CV 2/5] END alpha=0.0001, copy_X=False, fit_intercept=True, selection=cyclic, tol=0.1, warm_start=False;, score=-1.320 total time=   0.1s
[CV 4/5] END alpha=0.0001, copy_X=False, fit_intercept=True, selection=cyclic, tol=0.1, warm_start=False;, score=-1.138 total time=   0.3s
[CV 3/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.0001, warm_start=True;, score=-1.194 total time=   0.1s
[CV 4/5] END alpha=0.01, copy_X=False, fit_intercept=False, selection=cyclic, tol=0.0001, warm_start=False;, score=-1.160 total time=   0.2s
[CV 1/5] END alpha=10, co

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.618e+05, tolerance: 1.605e+05
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.323e+05, tolerance: 1.605e+02
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

[CV 3/5] END alpha=0.0001, copy_X=False, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.195 total time=   0.2s
[CV 5/5] END alpha=0.0001, copy_X=False, fit_intercept=True, selection=cyclic, tol=0.1, warm_start=False;, score=-1.066 total time=   0.3s
[CV 1/5] END alpha=10, copy_X=True, fit_intercept=False, selection=random, tol=1, warm_start=True;, score=-3.051 total time=   0.0s
[CV 2/5] END alpha=10, copy_X=True, fit_intercept=False, selection=random, tol=1, warm_start=True;, score=-1.384 total time=   0.0s
[CV 5/5] END alpha=0.0001, copy_X=False, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.066 total time=   0.3s
[CV 1/5] END alpha=1, copy_X=True, fit_intercept=True, selection=random, tol=1, warm_start=False;, score=-1.467 total time=   0.0s
[CV 2/5] END alpha=1, copy_X=True, fit_intercept=True, selection=random, tol=1, warm_start=False;, score=-1.335 total time=   0.0s


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Lasso(), n_iter=25, n_jobs=-1,
                   param_distributions={'alpha': [0.0001, 0.001, 0.01, 0.1, 1,
                                                  10],
                                        'copy_X': [True, False],
                                        'fit_intercept': [True, False],
                                        'selection': ['cyclic', 'random'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1, 1,
                                                10],
                                        'warm_start': [True, False]},
                   random_state=42,
                   scoring=make_scorer(mean_absolute_scaled_error, greater_is_better=False),
                   verbose=3)

In [11]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)
print('Best Model: %s' % search.best_estimator_)
print('Best Index: %s' % search.best_index_)
print('CV Results: %s' % search.cv_results_)
print('Refit Time: %s' % search.refit_time_)
print('Scorer: %s' % search.scorer_)

Best Score: -1.1800154766869375
Best Hyperparameters: {'warm_start': True, 'tol': 0.0001, 'selection': 'random', 'fit_intercept': False, 'copy_X': True, 'alpha': 0.01}
Best Model: Lasso(alpha=0.01, fit_intercept=False, selection='random', warm_start=True)
Best Index: 23
CV Results: {'mean_fit_time': array([0.12586174, 0.00692544, 0.00631819, 0.006319  , 0.00627899,
       0.00620737, 0.00788684, 0.20902634, 0.00855598, 0.00947537,
       0.01753173, 0.01098394, 0.09904566, 0.00850029, 0.0075129 ,
       0.00797987, 0.08633814, 0.21887779, 0.00726595, 0.01173487,
       0.01351523, 0.22546587, 0.0099998 , 0.13067398, 0.00928621]), 'std_fit_time': array([0.05109531, 0.00251248, 0.00247577, 0.00220247, 0.00225969,
       0.00222643, 0.00291912, 0.127421  , 0.00491472, 0.00404123,
       0.00903823, 0.00544423, 0.05027675, 0.00525053, 0.00326024,
       0.00439734, 0.06491659, 0.09934061, 0.00331167, 0.00911425,
       0.00688512, 0.09132941, 0.0066566 , 0.05760919, 0.0052464 ]), 'mean_sco